## Project Setup

In [ ]:
# install needed libraries
"""!pip install pyarrow
!pip install fastparquet
!pip install geopandas
!pip install pytest
!pip install keplergl
!pip install bs4"""

# all import statements needed for the project, for example:
import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import sqlite3
import numpy as np
import matplotlib.animation as animation
import keplergl
from keplergl import KeplerGl
import statistics
import unittest
import os

In [ ]:
# add constant
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

# Part 3: Understanding the Data

In [ ]:
def write_queries_to_files(query_dict: dict):
    # Create the directory if it does not exist
    if not os.path.exists(QUERY_DIRECTORY):
        os.makedirs(QUERY_DIRECTORY)

    # Iterate over the dictionary items and write each query to a file
    for query_name, query_string in query_dict.items():
        # Construct the file path using the query directory and query name
        file_path = os.path.join(QUERY_DIRECTORY, f"{query_name}.sql")
        
        # Write the query string to the file
        with open(file_path, "w") as f:
            f.write(query_string)